# CHOIX DE LA TAILLE

In [1]:
size = 120

# SCRIPT

In [12]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

from skimage import data

from PIL.Image import *

from imageio import imread
import glob
from tqdm import tnrange
from tqdm.notebook import tqdm

matplotlib.rcParams['font.size'] = 18

In [13]:
%run preprocessing

In [27]:
list_images = []
list_labels = []

for image_path in tqdm(glob.glob("test/*.png"), total = len(glob.glob("test/*.png"))):
    image = imread(image_path)
    list_images.append(image)
    debut = 5
    fin = image_path.find('i')
    list_labels.append(int(image_path[debut:fin]))

KeyboardInterrupt: 

In [28]:
num_classes = np.max(list_labels)+1

In [29]:
def rescale_reshape(list_images, size):
    for i in tnrange(len(list_images)) :
        img = list_images[i]
        img = to_float32(img)
        img = rescale(img,size, size)
        list_images[i] = img

In [30]:
rescale_reshape(list_images, size)

C:\Users\stani\anaconda3\envs\modal\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  


In [32]:
def ajout_transfo(list_images, high1=0.5, low1=0.1, high2=0.2, low2=0.05) :
    for i in tnrange(len(list_images)) :
        img = list_images[i]
        r,g,b = rgb(img)
        print(r)
        bandw = rgb_to_bandw(img)
        vis_grad_g, vis_grad_b = vis_grad(bandw)
        vis_hessian_g, vis_hessian_b = vis_hessian(bandw)
        list_images[i] = [r,g,b,
                          vis_grad_g, vis_grad_b,
                          canny_edge_detection(bandw, high1, low1),
                          canny_edge_detection(bandw, high2, low2),
                          vis_hessian_g, vis_hessian_b]

In [33]:
ajout_transfo(list_images)
deep = 9

C:\Users\stani\anaconda3\envs\modal\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  


[[[0.51372552]
  [0.37751636]
  [0.49934641]
  ...
  [0.        ]
  [0.        ]
  [0.01176471]]

 [[0.46431375]
  [0.40716341]
  [0.51827455]
  ...
  [0.01552941]
  [0.        ]
  [0.01359477]]

 [[0.56078434]
  [0.59769934]
  [0.91006535]
  ...
  [0.14462745]
  [0.04847059]
  [0.02128105]]

 ...

 [[0.24156864]
  [0.16308497]
  [0.17385621]
  ...
  [0.11670589]
  [0.29908499]
  [0.13798693]]

 [[0.13725491]
  [0.19090196]
  [0.0997647 ]
  ...
  [0.24360785]
  [0.3272157 ]
  [0.18839216]]

 [[0.15686275]
  [0.20366013]
  [0.24141176]
  ...
  [0.27529413]
  [0.37458825]
  [0.31325492]]]
[[[0.03137255]
  [0.20784314]
  [0.01176471]
  ...
  [0.03921569]
  [0.78169936]
  [0.1006536 ]]

 [[0.01568628]
  [0.0627451 ]
  [0.03006536]
  ...
  [0.        ]
  [0.49673203]
  [0.08366013]]

 [[0.01568628]
  [0.04444445]
  [0.0248366 ]
  ...
  [0.07450981]
  [0.03006536]
  [0.15424837]]

 ...

 [[0.13725491]
  [0.22091503]
  [0.2379085 ]
  ...
  [0.07058824]
  [0.08235294]
  [0.08888889]]

 [[0.254

KeyboardInterrupt: 

In [20]:
# NE PAS APPELER vis_hessian A PARTIR DE MTN


import torch
from torchvision import datasets,transforms
import torch.nn.functional as F
import torch.nn as nn
from torchvision.utils import save_image

In [21]:
def list_to_torch(list_images, list_labels, size, deep) :
    n = len(list_images)
    
    torch_images=np.array([np.array(data) for data in list_images])
    torch_labels=np.array([np.array(data) for data in list_labels])
    
    torch_images = torch_images.reshape([n,deep,size,size])
    torch_labels = torch_labels.reshape([n])
    
    torch_images = torch.from_numpy(torch_images)
    torch_labels = torch.from_numpy(torch_labels)
    
    train_set = torch.utils.data.TensorDataset(torch_images[:int(0.8*n)],torch_labels[:int(0.8*n)])
    test_set = torch.utils.data.TensorDataset(torch_images[int(0.8*n):],torch_labels[int(0.8*n):])
    
    return train_set, test_set

In [22]:
train_set, test_set = list_to_torch(list_images, list_labels, size, deep)

## S'il y a des images avec de mauvaises shape

In [25]:
"""
for num_image in range(len(list_images)) :
    for i in range(9) :
        if list_images[num_image][i].shape != (120,120,1) :
            print(num_image, i)
"""

In [24]:
"""
del list_images[2929]
del list_labels[2929]
"""

In [24]:
compteur_im = 0
data = torch.utils.data.DataLoader(train_set, batch_size=1)
for image, label in data :
    image = image[0]
    label = int(label[0])
    for couche in range(deep) :
        print(image[couche])
        print(image[couche].shape)
        save_image(image[couche], 'test24/'+ str(compteur_im) + 'l' + str(label) + 'c' + str(couche) +'.png')
    compteur_im +=1

data = torch.utils.data.DataLoader(test_set, batch_size=1)
for image, label in data :
    image = image[0]
    label = int(label[0])
    for couche in range(9) :
        save_image(image[couche], 'test24/'+ str(compteur_im) + 'l' + str(label) + 'c' + str(couche) +'.png')
    compteur_im +=1

tensor([[0.5137, 0.3775, 0.4993,  ..., 0.0000, 0.0000, 0.0118],
        [0.4643, 0.4072, 0.5183,  ..., 0.0155, 0.0000, 0.0136],
        [0.5608, 0.5977, 0.9101,  ..., 0.1446, 0.0485, 0.0213],
        ...,
        [0.2416, 0.1631, 0.1739,  ..., 0.1167, 0.2991, 0.1380],
        [0.1373, 0.1909, 0.0998,  ..., 0.2436, 0.3272, 0.1884],
        [0.1569, 0.2037, 0.2414,  ..., 0.2753, 0.3746, 0.3133]],
       dtype=torch.float64)
torch.Size([120, 120])
tensor([[0.7294, 0.5522, 0.6675,  ..., 0.0314, 0.0159, 0.0196],
        [0.5812, 0.5850, 0.6815,  ..., 0.0143, 0.0220, 0.0214],
        [0.7106, 0.6947, 0.8997,  ..., 0.3046, 0.0232, 0.0291],
        ...,
        [0.4337, 0.3209, 0.3127,  ..., 0.0971, 0.2092, 0.0964],
        [0.2596, 0.3579, 0.2645,  ..., 0.2430, 0.2409, 0.1483],
        [0.3137, 0.3608, 0.4249,  ..., 0.2466, 0.2705, 0.2584]],
       dtype=torch.float64)
torch.Size([120, 120])
tensor([[4.0392e-01, 3.2183e-01, 3.8719e-01,  ..., 0.0000e+00, 1.5948e-02,
         1.5686e-02],
     

       dtype=torch.float64)
torch.Size([120, 120])
tensor([[0.0000, 0.0000, 0.0567,  ..., 0.0106, 0.0000, 0.0832],
        [0.0758, 0.0392, 0.0000,  ..., 0.0180, 0.0056, 0.0004],
        [0.0000, 0.0073, 0.0278,  ..., 0.0000, 0.0044, 0.0084],
        ...,
        [0.0510, 0.0232, 0.0314,  ..., 0.0223, 0.0467, 0.0510],
        [0.0353, 0.1139, 0.0293,  ..., 0.1121, 0.1826, 0.1885],
        [0.0366, 0.0297, 0.0403,  ..., 0.1346, 0.0808, 0.2200]],
       dtype=torch.float64)
torch.Size([120, 120])
tensor([[-0.3145, -0.4504, -0.0042,  ..., -0.0032,  0.3242,  0.0204],
        [-0.2235, -0.3954, -0.2387,  ..., -0.2597,  0.0193,  0.1465],
        [-0.0541, -0.1650, -0.1298,  ..., -0.1737, -0.1005,  0.0882],
        ...,
        [ 0.0881, -0.0879, -0.0749,  ..., -0.4357,  0.0861,  0.0195],
        [ 0.2505, -0.1586, -0.2756,  ..., -0.6286,  0.2744,  0.2043],
        [ 0.1158, -0.1785, -0.0652,  ..., -0.4034,  0.3078,  0.5158]],
       dtype=torch.float64)
torch.Size([120, 120])
tensor([[-0.026

       dtype=torch.float64)
torch.Size([120, 120])
tensor([[ 0.0884,  0.0684,  0.0601,  ...,  0.1403,  0.1308,  0.1329],
        [-0.2152, -0.0046,  0.0735,  ..., -0.3242, -0.5545, -0.1831],
        [-1.2985, -1.0507, -0.9095,  ..., -0.2046, -0.0667,  0.0481],
        ...,
        [-0.6473, -0.4671, -0.4782,  ...,  0.5525,  0.5213,  0.8473],
        [-0.4864, -0.5864, -0.4358,  ...,  0.6753,  0.7482,  0.5517],
        [-0.0092, -0.0880, -0.0597,  ...,  0.1261,  0.0346, -0.3182]],
       dtype=torch.float64)
torch.Size([120, 120])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)
torch.Size([120, 120])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 0., 0.

       dtype=torch.float64)
torch.Size([120, 120])
tensor([[1.8207e-02, 1.1542e-01, 5.0553e-02,  ..., 1.6861e-02, 7.7037e-03,
         2.5339e-02],
        [1.5873e-02, 5.3350e-02, 3.1342e-02,  ..., 4.9314e-02, 7.9243e-02,
         6.1027e-02],
        [2.4009e-02, 1.1810e-01, 1.8478e-01,  ..., 3.3351e-02, 1.8422e-01,
         3.4307e-04],
        ...,
        [2.6050e-03, 1.4543e-01, 1.2931e-02,  ..., 4.3545e-01, 4.0375e-01,
         1.2668e-02],
        [3.8426e-02, 1.1872e-02, 5.4245e-02,  ..., 2.1822e-05, 3.1148e-01,
         1.5958e-01],
        [1.5256e-04, 3.2225e-02, 3.3440e-02,  ..., 2.4990e-01, 6.3470e-02,
         1.0092e-01]], dtype=torch.float64)
torch.Size([120, 120])
tensor([[0.2784, 0.2837, 0.2614,  ..., 0.2157, 0.0967, 0.1033],
        [0.1922, 0.2810, 0.2314,  ..., 0.4431, 0.1386, 0.1268],
        [0.2863, 0.2627, 0.2353,  ..., 0.1333, 0.1739, 0.1660],
        ...,
        [0.7490, 0.5477, 0.2954,  ..., 0.2980, 0.3830, 0.4510],
        [0.3608, 0.3686, 0.5085,  ..., 0

         3.6395e-02]], dtype=torch.float64)
torch.Size([120, 120])
tensor([[0.1176, 0.2170, 0.1569,  ..., 0.0941, 0.3529, 0.2405],
        [0.2314, 0.2771, 0.2039,  ..., 0.1098, 0.1451, 0.1778],
        [0.0902, 0.2745, 0.3804,  ..., 0.3451, 0.1948, 0.3804],
        ...,
        [0.8314, 0.7190, 0.7516,  ..., 0.6039, 0.6458, 0.6784],
        [0.7176, 0.8065, 0.9072,  ..., 0.5961, 0.7190, 0.5634],
        [0.8627, 0.7242, 0.6824,  ..., 0.5294, 0.5752, 0.8039]],
       dtype=torch.float64)
torch.Size([120, 120])
tensor([[0.1686, 0.2523, 0.1961,  ..., 0.1765, 0.4771, 0.3425],
        [0.2824, 0.3124, 0.2431,  ..., 0.2078, 0.2431, 0.2928],
        [0.1294, 0.2954, 0.4039,  ..., 0.4392, 0.3203, 0.4928],
        ...,
        [0.8039, 0.6915, 0.7242,  ..., 0.5412, 0.6065, 0.6340],
        [0.6863, 0.7752, 0.8719,  ..., 0.5451, 0.6797, 0.5163],
        [0.8353, 0.6967, 0.6549,  ..., 0.4627, 0.5203, 0.7399]],
       dtype=torch.float64)
torch.Size([120, 120])
tensor([[0.1333, 0.2183, 0.1529,  .

...

          5.1681e-02,  1.3420e-03]], dtype=torch.float64)
torch.Size([120, 120])
tensor([[0.1059, 0.0641, 0.0170,  ..., 0.1137, 0.2954, 0.1582],
        [0.0353, 0.0614, 0.3529,  ..., 0.0980, 0.1046, 0.1804],
        [0.1216, 0.0850, 0.1229,  ..., 0.0431, 0.0863, 0.3255],
        ...,
        [0.3059, 0.3765, 0.2458,  ..., 0.5686, 0.5673, 0.6131],
        [0.3020, 0.2719, 0.1621,  ..., 0.3882, 0.5882, 0.6131],
        [0.2627, 0.2654, 0.1490,  ..., 0.2706, 0.5582, 0.5830]],
       dtype=torch.float64)
torch.Size([120, 120])
tensor([[0.1529, 0.1634, 0.0680,  ..., 0.1686, 0.4013, 0.1791],
        [0.0863, 0.1163, 0.3987,  ..., 0.1412, 0.1438, 0.2484],
        [0.1373, 0.1307, 0.1542,  ..., 0.0863, 0.1255, 0.3922],
        ...,
        [0.3098, 0.3804, 0.2497,  ..., 0.5529, 0.5556, 0.6092],
        [0.3216, 0.2915, 0.1974,  ..., 0.3725, 0.5843, 0.6131],
        [0.2745, 0.2967, 0.1686,  ..., 0.2510, 0.5582, 0.5830]],
       dtype=torch.float64)
torch.Size([120, 120])
tensor([[0.1059, 0.03

       dtype=torch.float64)
torch.Size([120, 120])
tensor([[-1.0510e-01, -9.7759e-01, -1.2304e+00,  ...,  2.1922e-01,
          1.2392e-01, -7.2405e-02],
        [-6.7346e-02, -4.0708e-01, -4.8863e-01,  ...,  1.7098e-01,
          1.2084e-01, -2.9830e-02],
        [-6.6889e-02,  1.1799e-01,  1.5267e-01,  ...,  2.9471e-01,
          9.6575e-02, -1.1342e-01],
        ...,
        [ 5.5294e-03,  7.8301e-03,  1.1922e-02,  ...,  3.5098e-02,
          1.8993e-02, -1.8052e-02],
        [ 2.2353e-03, -3.0065e-04,  3.7255e-03,  ...,  1.7477e-02,
         -6.1830e-03, -1.8314e-02],
        [ 1.1765e-03,  3.0458e-03,  0.0000e+00,  ...,  3.3464e-03,
         -1.3673e-02, -1.7503e-02]], dtype=torch.float64)
torch.Size([120, 120])
tensor([[ 1.2927e+00,  1.0425e+00,  4.8586e-01,  ...,  5.7516e-04,
          6.1203e-02, -3.8039e-03],
        [ 1.6748e+00,  1.3167e+00,  5.6105e-01,  ..., -1.3349e-01,
         -1.4280e-01, -1.1728e-01],
        [-3.0529e-01, -4.7273e-01, -8.8346e-01,  ..., -2.0540e-01,


        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)
torch.Size([120, 120])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 1., 1., 1.],
        [0., 1., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)
torch.Size([120, 120])
tensor([[0.0271, 0.0225, 0.0401,  ..., 0.0755, 0.1612, 0.2419],
        [0.0164, 0.0735, 0.0812,  ..., 0.0940, 0.1238, 0.3669],
        [0.0303, 0.2143, 0.3674,  ..., 0.1800, 0.0975, 0.2410],
        ...,
        [0.0436, 0.3370, 0.2561,  ..., 0.0710, 0.2994, 0.0941],
        [0.1480, 0.4574, 0.2554,  ..., 0.2492, 0.1447, 0.1731],
        [0.2938, 0.1652, 0.0216,  ..., 0.2216, 0.0013, 0.2472]],
       dtype=torch.float64)
torch.Size([120, 120])
tensor([[0.0262, 0.0653, 0.0365,  ..., 0.0432, 0.1679, 0.1121],
        [0.0028, 0.0028, 0.0278,  ..., 0.0456, 0.0500, 0.0031],
        [0.0026, 0.0839, 0.0024,  ..., 0

       dtype=torch.float64)
torch.Size([120, 120])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)
torch.Size([120, 120])
tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.]], dtype=torch.float64)
torch.Size([120, 120])
tensor([[0.0517, 0.0440, 0.0918,  ..., 0.0950, 0.0535, 0.0269],
        [0.0109, 0.1153, 0.0027,  ..., 0.0770, 0.1282, 0.0254],
        [0.0365, 0.0911, 0.0623,  ..., 0.0586, 0.0850, 0.0348],
        ...,
        [0.1567, 0.1851, 0.2275,  ..., 0.3250, 0.1132, 0.3129],
        [0.1735, 0.1236, 0.1545,  ..., 0.1092, 0.1254, 0.2606],
        [0.0832, 0.0

KeyboardInterrupt: 

In [32]:
len(list_images)

7181

In [30]:
compteur_im

7181

# stockage image noir et blanc

In [19]:
image = imread("neb/11118image209.png")

In [25]:
debut = 4
fin = "neb/11118image209.png".find('i')
label = int("neb/11118image209.png"[debut:fin])

In [26]:
label

11118

In [27]:
img = image
r,g,b = rgb(img) 
bandw = rgb_to_bandw(img)
vis_grad_g, vis_grad_b = vis_grad(bandw)
vis_hessian_g, vis_hessian_b = vis_hessian(bandw)

In [30]:
b.shape

(349, 512, 1)